# Quiz
MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Oscar Javier Angel Balcázar

In [57]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from plotly.offline import init_notebook_mode, iplot, plot
import matplotlib.pyplot as plt
import seaborn as sns


### Reading the dataset

In [2]:
medicalCost_df = pd.read_csv('https://raw.githubusercontent.com/ojangelb/MINE-4101/main/Quiz/insurance.csv', sep = ',')

In [3]:
medicalCost_df.shape

(1338, 7)

Se eliminan **duplicados**

In [4]:
medicalCost_df.drop_duplicates()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### Features to train the model

In [5]:
medicalCost_df['sex'] = medicalCost_df['sex'].replace('male',1)
medicalCost_df['sex'] = medicalCost_df['sex'].replace('female',0)
medicalCost_df['smoker'] = medicalCost_df['smoker'].replace('yes',1)
medicalCost_df['smoker'] = medicalCost_df['smoker'].replace('no',0)

Regiones

In [6]:
medicalCost_df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [7]:
medicalCost_df['region'] = medicalCost_df['region'].replace('southwest',1)
medicalCost_df['region'] = medicalCost_df['region'].replace('southeast',2)
medicalCost_df['region'] = medicalCost_df['region'].replace('northwest',3)
medicalCost_df['region'] = medicalCost_df['region'].replace('northeast',4)

### División del datasets

In [8]:
features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
X = medicalCost_df[features]
Y = medicalCost_df['charges']

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [10]:
X_train.shape

(1070, 6)

In [11]:
X_test.shape

(268, 6)

### Scaling features

In [12]:
scaler = StandardScaler()

In [13]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Entrenamiento del Modelo - Regresión Lineal

In [14]:
regr = LinearRegression()

In [15]:
regr.fit(X_train_scaled, Y_train)

LinearRegression()

In [16]:
regr.coef_

array([3602.77541857, -122.13671165, 1929.20311613,  497.80337376,
       9605.29975404,  367.95675551])

In [17]:
regr.intercept_

13230.161574933647

### Evaluación del modelo

In [18]:
preds_train = regr.predict(X_train_scaled)
preds_test = regr.predict(X_test_scaled)

In [19]:
mean_absolute_error(Y_train, preds_train), mean_absolute_error(Y_test, preds_test)

(4170.334007230732, 4056.1183056626537)

In [20]:
np.sqrt(mean_squared_error(Y_train, preds_train)), np.sqrt(mean_squared_error(Y_test, preds_test))

(6067.7734770582765, 5956.564766052585)

1. ¿Es aceptable el error obtenido?

De acuerdo a los resultados el modelo esta prediciendo los valores de cada seguro en más de 6.000 dólares.

¿Hay evidencia de overfitting?

No, como se mencionó anteriormente el error en prueba y en test es muy alto, es decir hay underfitting. 

### Entrenamiento del Modelo - Transformación polinomial

In [21]:
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

poly_reg = LinearRegression()

X_train_poly, X_test_poly, Y_train_poly, Y_test_poly = train_test_split(X_poly, Y, test_size = 0.2, random_state = 1)

poly_reg.fit(X_train_poly, Y_train_poly)

LinearRegression()

Evaluación del modelo

In [22]:
poly_reg.coef_

array([-6.24297125e+01, -9.38117865e+02,  3.69414666e+02,  1.06528973e+03,
       -8.95881680e+03, -6.44649831e+02,  3.79521582e+00,  2.52888916e+01,
        1.65193405e+00,  3.21901963e+00, -2.04577457e+01, -1.63211492e+01,
       -9.38117865e+02,  2.41943663e+01, -2.23940686e+02,  2.52419140e+01,
       -1.05309732e+02, -7.89016402e+00, -1.20474370e+01,  1.43104538e+03,
        3.00592907e+01, -9.13892065e+01, -4.75994689e+02,  1.33290544e+02,
       -8.95881680e+03, -3.26472106e+02,  1.55208406e+02])

In [23]:
poly_reg.intercept_

-2261.7532508366476

In [24]:
preds_train_poly = poly_reg.predict(X_train_poly)
preds_test_poly = poly_reg.predict(X_test_poly)

In [25]:
np.sqrt(mean_squared_error(Y_train_poly, preds_train_poly)), np.sqrt(mean_squared_error(Y_test_poly, preds_test_poly))

(4795.25618007516, 4547.832132093696)

Trasformación Polinomial Grado 11

In [26]:
poly_features_11 = PolynomialFeatures(degree=11, include_bias=False)
X_poly_11 = poly_features_11.fit_transform(X)

poly_reg_11 = LinearRegression()

X_train_poly_11, X_test_poly_11, Y_train_poly_11, Y_test_poly_11 = train_test_split(X_poly_11, Y, test_size = 0.2, random_state = 1)

poly_reg_11.fit(X_train_poly_11, Y_train_poly_11)

LinearRegression()

In [27]:
poly_reg_11.coef_

array([-2.07770061e-02, -1.47819491e-03, -1.53623179e-03, ...,
       -3.86447722e-07, -1.50857081e-06, -9.13619728e-04])

In [28]:
poly_reg_11.intercept_

15962.84944495928

In [29]:
preds_train_poly_11 = poly_reg_11.predict(X_train_poly_11)
preds_test_poly_11 = poly_reg_11.predict(X_test_poly_11)

In [30]:
np.sqrt(mean_squared_error(Y_train_poly_11, preds_train_poly_11)), np.sqrt(mean_squared_error(Y_test_poly_11, preds_test_poly_11))

(1456.3740183435718, 21319088.092977304)

Regularización Ridge

Alfa = 0.001

In [74]:
X_train_ridge, X_test_ridge, Y_train_ridge, Y_test_ridge = train_test_split(X_poly_11, Y, test_size = 0.2, random_state = 1)

reg_ridge = SGDRegressor(alpha=0.001, penalty="L2", tol = None, max_iter= 1000)

reg_ridge.fit(X_train_ridge, Y_train_ridge)


SGDRegressor(alpha=0.001, penalty='L2', tol=None)

In [59]:
reg_ridge.coef_

array([ 6.25729231e+12,  1.22953283e+11,  7.93913215e+12, ...,
        2.25112260e+14,  9.32445207e+14, -2.93367648e+17])

In [60]:
reg_ridge.intercept_

array([1.67598015e+12])

In [61]:
preds_train_reg = reg_ridge.predict(X_train_ridge)
preds_test_reg = reg_ridge.predict(X_test_ridge)

In [62]:
np.sqrt(mean_squared_error(Y_train_ridge, preds_train_reg)), np.sqrt(mean_squared_error(Y_test_ridge, preds_test_reg))

(1.6026204052378308e+47, 1.6090706063570007e+47)

Alfa = 0.1

In [64]:
X_train_ridge_1, X_test_ridge_1, Y_train_ridge_1, Y_test_ridge_1 = train_test_split(X_poly_11, Y, test_size = 0.2, random_state = 1)

reg_ridge_1 = SGDRegressor(alpha=0.1, penalty="L2", tol = None, max_iter= 1000)

reg_ridge_1.fit(X_train_ridge_1, Y_train_ridge_1)


SGDRegressor(alpha=0.1, penalty='L2', tol=None)

In [65]:
reg_ridge_1.coef_

array([-1.32906599e+11, -6.29013692e+09,  3.02464018e+11, ...,
       -9.52351157e+14, -3.77598508e+15, -1.05538030e+17])

In [66]:
reg_ridge_1.intercept_

array([-4.81931632e+11])

In [67]:
preds_train_reg_1 = reg_ridge_1.predict(X_train_ridge_1)
preds_test_reg_1 = reg_ridge_1.predict(X_test_ridge_1)

In [68]:
np.sqrt(mean_squared_error(Y_train_ridge_1, preds_train_reg_1)), np.sqrt(mean_squared_error(Y_test_ridge_1, preds_test_reg_1))

(1.5841631501885158e+47, 1.6082280102600318e+47)

Alfa = 2

In [69]:
X_train_ridge_2, X_test_ridge_2, Y_train_ridge_2, Y_test_ridge_2 = train_test_split(X_poly_11, Y, test_size = 0.2, random_state = 1)

reg_ridge_2 = SGDRegressor(alpha=0.1, penalty="L2", tol = None, max_iter= 1000)

reg_ridge_2.fit(X_train_ridge_2, Y_train_ridge_2)


SGDRegressor(alpha=0.1, penalty='L2', tol=None)

In [70]:
reg_ridge_2.coef_

array([-2.89864627e+12, -1.12904299e+10, -1.75558879e+12, ...,
        6.47084666e+14,  2.40481004e+15, -1.47412350e+17])

In [71]:
reg_ridge_2.intercept_

array([1.1285965e+12])

In [72]:
preds_train_reg_2 = reg_ridge_2.predict(X_train_ridge_2)
preds_test_reg_2 = reg_ridge_2.predict(X_test_ridge_2)

In [73]:
np.sqrt(mean_squared_error(Y_train_ridge_2, preds_train_reg_2)), np.sqrt(mean_squared_error(Y_test_ridge_2, preds_test_reg_2))

(3.590260529016376e+47, 3.627680262449805e+47)

¿Fue posible mejorar el error? ¿Qué hiper-parámetros tiene el modelo que produce el menor error?

No, no fue posible disminuir el error, se observó que al aumentar el valor de alfa aumentaba mucho el error.

